<a href="https://colab.research.google.com/github/NuarkNoir/UPC/blob/master/4sem/isaip/02/RSA/rsaNk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Алгоритм шифрования RSA

In [ ]:
import random

def n_bit_random(n):
    return(random.randrange(2**(n-1)+1, 2**n-1))

first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 
                     53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 
                     109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 
                     173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 
                     233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 
                     293, 307, 311, 313, 317, 331, 337, 347, 349]

def check_low_level_prime(n):
    while True: 
        prime_candidate = n_bit_random(n)  
        for divisor in first_primes_list:  
            if prime_candidate % divisor == 0 and divisor**2 <= prime_candidate: 
                break
            return prime_candidate 

def check_miller_rabin_primality(mrc):
    max_divisions_by_two = 0
    ec = mrc-1
    while ec % 2 == 0: 
        ec >>= 1
        max_divisions_by_two += 1
    assert(2**max_divisions_by_two * ec == mrc-1) # lame
  
    def trial_composite(round_tester): 
        if pow(round_tester, ec, mrc) == 1: 
            return False
        for i in range(max_divisions_by_two): 
            if pow(round_tester, 2**i * ec, mrc) == mrc-1: 
                return False
        return True
  
    trials_amount = 20 
    for i in range(trials_amount): 
        round_tester = random.randrange(2, mrc) 
        if trial_composite(round_tester): 
            return False
    return True

def n_bit_random_prime(n):
    while True:
        possible_prime = check_low_level_prime(n) 
        if not check_miller_rabin_primality(possible_prime): 
            continue
        return possible_prime

In [ ]:
import math

import decimal
decimal.getcontext().prec=10000
_d = lambda x: decimal.Decimal(x)

# Modular exponentiation goes brrrrr 
def modular_pow(base, exponent, modulus):
    base, modulus = _d(base), _d(modulus)
    base %= modulus
    result = 1
    while exponent > 0:
        if int(exponent) & 1:
            result = (result * base) % modulus
        base = (base * base) % modulus
        exponent //= 2
    return result

def encrypt(message, exponent, modulus):
    out = []
    for c in message:
        encv = modular_pow(ord(c), exponent, modulus)
        out.append(encv)
    return out

def decrypt(encrypted, pkey, modulus):
    out = []
    for num in encrypted:
        decv = modular_pow(num, pkey, modulus)
        out.append(chr(decv))
    return "".join(out)

def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)

def choose_e(lcmv):
    while 1:
        e = random.randrange(3, 2**16+2)
        if e < lcmv and math.gcd(e, lcmv) == 1:
            return e

def extended_gcd(a, b):
    x, old_x = 0, 1
    y, old_y = 1, 0

    while (b != 0):
        quotient = a // b
        a, b = b, a - quotient * b
        old_x, x = x, old_x - quotient * x
        old_y, y = y, old_y - quotient * y

    return a, old_x, old_y

In [ ]:
key_length = 1024
%time P = n_bit_random_prime(key_length)
%time Q = n_bit_random_prime(key_length)

CPU times: user 3.67 s, sys: 999 µs, total: 3.67 s
Wall time: 3.69 s
CPU times: user 1.85 s, sys: 0 ns, total: 1.85 s
Wall time: 1.86 s


In [ ]:
n = P * Q
lcmv = lcm(P-1, Q-1)
e = choose_e(lcmv)

gcd, x, y = extended_gcd(e, lcmv)
if x < 0:
    d = x + lcmv
else:
    d = x

In [ ]:
print(e, d, n, sep="\n")

39649
2775085486572169457933031624471639527709572371571563930751706363455686679103931887012786872357300821852072462055701332168753464477069850100477857634950881057962034588705265582670348431618430187943675082038338808823502290083334757934315038640497969786069367822629393372740634217481080439281756759203535191518030611113366713687329262086331416243914608547464393822243234244247787598602422663859280454972323707900579310354345134163802897882709119063046579316638215239586955495026262129316401981036727648176233721051089840544139490182413169518916207283731389315379326286341277949065063422651145928466610428684652315668195
23566779236161770643112862787707714768319744752887788874531340221608793304492584597473313112218174480194085827590012234139629339174956445928785901695962763659830088046458803074421870909628916257832886158798721043310543454720481166030983536298954914657023031763702456252981131493715209010905445417269332126191376334319895522288189118297669317432037971085289460034222690931966559

## Тестирование алгоритма шифрования

In [ ]:
message = "sosait"
encrypted_message = encrypt(message, e, n)
decrypted_message = decrypt(encrypted_message, d, n)

print(message, "Encrypted message will be really long", decrypted_message, sep="\n")

sosait
Encrypted message will be really long
sosait


In [ ]:
import os
import pickle
import IPython
from google.colab import output, files

def prepare_file(filename, data):
    with open(filename, "wb+") as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    return filename

def download_public():
    files.download(prepare_file("public.key", [e, n]))

def download_private():
    files.download(prepare_file("private.key", [d, n]))

def download_unlink(filename):
    try:
        os.remove(filename)
    except OSError:
        pass

output.register_callback("notebook.download_public", download_public)
output.register_callback("notebook.download_private", download_private)
output.register_callback("notebook.download_unlink", download_unlink)

In [ ]:
import os
import pickle
import IPython
from google.colab import output, files

def check_correctness():
    download_unlink("public.key")
    download_unlink("private.key")
    
    print("Select `public.key` and `private.key` files")
    uploaded = files.upload()
    
    eu, nf = pickle.loads(uploaded["public.key"])
    du, ns = pickle.loads(uploaded["private.key"])
    
    try:
        assert(nf == ns)
    except AssertionError:
        print("Keys modules are not equal!")
        return

    nu = nf

    test_message = "test messages for days"

    try:
        encm = encrypt(test_message, eu, nu)
        decm = decrypt(encm, du, nu)
    except AssertionError:
        print("Test message not correctly encrypted")
    else:
        print("Everything seems correct. Encryption and decryption worked well.")
    
output.register_callback("notebook.check_correctness", check_correctness)

In [ ]:
%%javascript
(async function() {
    let dPublic = document.createElement("button");
    let dPrivate = document.createElement("button");
    let dCheckC = document.createElement("button");

    dPublic.innerText = "Download public key";
    dPrivate.innerText = "Download private key";
    dCheckC.innerText = "Check Correctness";

    dPublic.addEventListener('click', async (event) => {
        await google.colab.kernel.invokeFunction("notebook.download_public", [], {});
        await google.colab.kernel.invokeFunction("notebook.download_unlink", ["public.key"], {});
    });

    dPrivate.addEventListener('click', async (event) => {
        await google.colab.kernel.invokeFunction("notebook.download_private", [], {});
        await google.colab.kernel.invokeFunction("notebook.download_unlink", ["private.key"], {});
    });

    dCheckC.addEventListener('click', async (event) => {
        await google.colab.kernel.invokeFunction("notebook.check_correctness", [], {});
    });

    document.querySelector("#output-area").appendChild(dPublic);
    document.querySelector("#output-area").appendChild(dPrivate);
    document.querySelector("#output-area").appendChild(dCheckC);
})();

<IPython.core.display.Javascript object>

Select `public.key` and `private.key` files


Saving private.key to private.key
Saving public.key to public.key
Everything seems correct. Encryption and decryption worked well.
